# Máster en Big Data Science - UNAV 
-----------------------------------------------------------------------------------------------------------------------------------------
### Técnicas de Recogida de datos - Proyecto - Deep wikipedia

## Aspectos generales:
- Objetivo: Mapa de Wikipedia: recuperar la página raíz y los primeros n enlaces (en azul) del primer párrafo del artículo.
- Página web: https://en.wikipedia.org/wiki/ y https://es.wikipedia.org/wiki/Main_Page
- Métodos utilizados: BeutifulSoup
- Resultado: Visualización en red

## Retos

Especificación de los aspectos que significaron mayor reto para el web scraping:
 - **API wikipedia:** Se investiga acerca del API propia de wikipedia, en donde se explora la extracción de los links asociados a otras páginas de wikipedia sin éxito. Se revisó la documentación en: https://www.mediawiki.org/wiki/API:Main_page/es   y   https://pypi.org/project/Wikipedia-API/
 - **Gráfico de red:** Se investiga la librería pyvis para lograr la visualización de red: https://pyvis.readthedocs.io/en/latest/. Esto solo permite una única visualización sin ser interactivo como en https://wikipedia.luk.ke/
 - **Distinto idioma:** Se realiza el mismo ejercicio para el idioma español con la dirección https://es.wikipedia.org/wiki/ y se identifica que la estructura de los artículos varía, por lo que el primer párrafo de contenido para el caso del idioma inglés es parrafo[1], mientras que para el idioma español es parrafo[0].

## 1. Importar librerías requeridas

In [1]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Librerías relacionadas con scraping
#Librería acceder a las páginas web con Python
import requests
#Librería para leer el HTML de la página web
from bs4 import BeautifulSoup

#Librerías relacionadas con la visualización
from pyvis.network import Network

## Búsqueda en inglés

### 2. Valor de la búsqueda en wikipedia en inglés

In [3]:
busqueda = input("Search Wikipedia: ")

Search Wikipedia:  cat


### 3. URL resultante para realizar la búsqueda

In [4]:
#Ajuste en caso de contener varias palabras
busqueda = busqueda.replace(" ","_")
#print(busqueda)

#url para scrapear
url = "https://en.wikipedia.org/wiki/"+str(busqueda)
print(url)

https://en.wikipedia.org/wiki/cat


### 4. Web scraping de la página de wikipedia resultante

In [5]:
#requests.get('página web') - de dónde se obtendrá la información
r = requests.get(url)
print(r.status_code)

#Contenido de todo el HTML
soup = BeautifulSoup(r.text,'html.parser')
#print(soup)

#Identificación de párrafos
parrafos = soup.find_all('p')
print(len(parrafos))
#Selección del primer párrafo de contenido
parrafo1 = parrafos[1]
print(parrafo1)

200
76
<p>The <b>cat</b> (<i>Felis catus</i>) is a <a href="/wiki/Domestication" title="Domestication">domestic</a> <a href="/wiki/Species" title="Species">species</a> of small <a class="mw-redirect" href="/wiki/Carnivorous" title="Carnivorous">carnivorous</a> <a href="/wiki/Mammal" title="Mammal">mammal</a>.<sup class="reference" id="cite_ref-Linnaeus1758_1-1"><a href="#cite_note-Linnaeus1758-1">[1]</a></sup><sup class="reference" id="cite_ref-MSW3fc_2-1"><a href="#cite_note-MSW3fc-2">[2]</a></sup> It is the only domesticated species in the family <a href="/wiki/Felidae" title="Felidae">Felidae</a> and is often referred to as the <b>domestic cat</b> to distinguish it from the wild members of the family.<sup class="reference" id="cite_ref-Clutton-Brock1999_4-0"><a href="#cite_note-Clutton-Brock1999-4">[4]</a></sup> A cat can either be a <b>house cat</b>, a <b><a href="/wiki/Farm_cat" title="Farm cat">farm cat</a></b> or a <b><a href="/wiki/Feral_cat" title="Feral cat">feral cat</a></b>

### 5. Identificación de links asociados al primer párrafo de contenido el artículo

#### 5.1 Extracción de todos los links

In [6]:
#Identificación de todos los links dentro del HTML del primer párrafo
links = parrafo1.find_all('a')
print(len(links))

15


#### 5.2 Links de referencia que son citas bibliográficas

In [7]:
#Identificación de referencias biliográficas en el párrafo
referencias = parrafo1.find_all('sup', attrs={'class':'reference'})
print(len(referencias))
bibliografia = []

for i in range (len(referencias)):
    referencia = referencias[i].find('a')
    link_ref = referencia.get('href')
    bibliografia.append(link_ref)

print(bibliografia)

5
['#cite_note-Linnaeus1758-1', '#cite_note-MSW3fc-2', '#cite_note-Clutton-Brock1999-4', '#cite_note-Liberg_al2014-5', '#cite_note-Driscoll_al2009-6']


#### 5.3 Enlaces hacia otras páginas de wikipedia

In [8]:
#Todos los liks y cuáles están asociados a bibliografía o a otros artículos de wikipedia que se encuentran en párrafo 1
titulos_ref = []

#Revisión de cada link
for i in range (len(links)):
    link = links[i].get('href')
    
    #En caso de ser link de referencia bibliográfica
    if link in bibliografia:
        #print('es una referencia bibliográfica')
        pass
    #En caso de ser link asociado a otro artículo de wikipedia
    else:
        titulo = links[i].get('title')
        #Almacenar el título del siguiente artículo
        titulos_ref.append(titulo)
print(titulos_ref)

['Domestication', 'Species', 'Carnivorous', 'Mammal', 'Felidae', 'Farm cat', 'Feral cat', 'Rodent', 'Cat breeds', 'Cat registries']


### 6. Visualización gráfica del resultado

In [9]:
#Definición de la red para ser visualizada en este notebook
net = Network(notebook=True)
#Agregar como nodo 1 la búsqueda inicial
net.add_node(1, label=busqueda) 
#Agregar los otros nodos relacionados a otros links de wikipedia
net.add_nodes(titulos_ref)

#Enlazar los artículos al artículo principal
for i in range (len(titulos_ref)):
    try:
        net.add_edge(1,titulos_ref[i])
    except:
        net.add_edge(1,int(titulos_ref[i]))

net.show("myred.html")

## Búsqueda en español

### 2. Valor de la búsqueda en wikipedia

In [10]:
busqueda = input("Búsqueda en Wikipedia: ")

Búsqueda en Wikipedia:  gato


### 3. URL resultante para realizar la búsqueda

In [11]:
#Ajuste en caso de contener varias palabras
busqueda = busqueda.replace(" ","_")
#print(busqueda)

#url para scrapear
url = "https://es.wikipedia.org/wiki/"+str(busqueda)
print(url)

https://es.wikipedia.org/wiki/gato


### 4. Web scraping de la página de wikipedia resultante

In [12]:
#requests.get('página web') - de dónde se obtendrá la información
r = requests.get(url)
print(r.status_code)

#Contenido de todo el HTML
soup = BeautifulSoup(r.text,'html.parser')
#print(soup)

#Identificación de párrafos
parrafos = soup.find_all('p')
print(len(parrafos))
#Selección del primer párrafo de contenido
parrafo1 = parrafos[0]
print(parrafo1)

200
125
<p>El <b>gato doméstico</b><sup class="reference separada" id="cite_ref-MSW3fc_1-0"><a href="#cite_note-MSW3fc-1"><span class="corchete-llamada">[</span>1<span class="corchete-llamada">]</span></a></sup>​<sup class="reference separada" id="cite_ref-ITIS_F.c._2-0"><a href="#cite_note-ITIS_F.c.-2"><span class="corchete-llamada">[</span>2<span class="corchete-llamada">]</span></a></sup>​ (<i><b>Felis silvestris catus</b></i>), llamado más comúnmente <b>gato</b>, y de forma coloquial <b>minino</b>,<sup class="reference separada" id="cite_ref-3"><a href="#cite_note-3"><span class="corchete-llamada">[</span>3<span class="corchete-llamada">]</span></a></sup>​ <b>michino</b>,<sup class="reference separada" id="cite_ref-4"><a href="#cite_note-4"><span class="corchete-llamada">[</span>4<span class="corchete-llamada">]</span></a></sup>​ <b>michi</b>,<sup class="reference separada" id="cite_ref-5"><a href="#cite_note-5"><span class="corchete-llamada">[</span>5<span class="corchete-llamada"

### 5. Identificación de links asociados al primer párrafo de contenido el artículo

#### 5.1 Extracción de todos los links

In [13]:
#Identificación de todos los links dentro del HTML del primer párrafo
links = parrafo1.find_all('a')
print(len(links))

15


#### 5.2 Links de referencia que son citas bibliográficas

In [14]:
#Identificación de referencias biliográficas en el párrafo
referencias = parrafo1.find_all('sup', attrs={'class':'reference'})
print(len(referencias))
bibliografia = []

for i in range (len(referencias)):
    referencia = referencias[i].find('a')
    link_ref = referencia.get('href')
    bibliografia.append(link_ref)

print(bibliografia)

10
['#cite_note-MSW3fc-1', '#cite_note-ITIS_F.c.-2', '#cite_note-3', '#cite_note-4', '#cite_note-5', '#cite_note-6', '#cite_note-7', '#cite_note-8', '#cite_note-9', '#cite_note-10']


#### 5.3 Enlaces hacia otras páginas de wikipedia

In [15]:
#Todos los liks y cuáles están asociados a bibliografía o a otros artículos de wikipedia que se encuentran en párrafo 1
titulos_ref = []

#Revisión de cada link
for i in range (len(links)):
    link = links[i].get('href')
    
    #En caso de ser link de referencia bibliográfica
    if link in bibliografia:
        #print('es una referencia bibliográfica')
        pass
    #En caso de ser link asociado a otro artículo de wikipedia
    else:
        titulo = links[i].get('title')
        #Almacenar el título del siguiente artículo
        titulos_ref.append(titulo)
print(titulos_ref)

['Mamífero', 'Carnívoro', 'Familia (biología)', 'Felidae', 'Homo sapiens']


### 6. Visualización gráfica del resultado los artículos al artículo principal

In [16]:
#Definición de la red para ser visualizada en este notebook
net = Network(notebook=True)
#Agregar como nodo 1 la búsqueda inicial
net.add_node(1, label=busqueda) 
#Agregar los otros nodos relacionados a otros links de wikipedia
net.add_nodes(titulos_ref)

#Enlazar los artículos al artículo principal
for i in range (len(titulos_ref)):
    try:
        net.add_edge(1,titulos_ref[i])
    except:
        net.add_edge(1,int(titulos_ref[i]))

net.show("myred.html")